### TABLAS DERIVADAS DE PRODUCTOS 50 Y 53 SE DEJAN EN SU PROPIA ETL DEBIDO A PROBLEMAS DE MEMORIA DE KERNEL JUPYTER 
(CORREGIR EN ORDEN DE ETLS)

In [2]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
from sqlalchemy import inspect
import sys
import sqlalchemy.ext.declarative as sqld
import roman

In [3]:
sys.path.append('..')

In [4]:
from modules.transforms import *

In [5]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [6]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [7]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [8]:
inspector = inspect(cnx)
schemas = inspector.get_schema_names()

In [9]:
tables={}
for schema in schemas:
    #print("schema: %s" % schema)
    tables[schema]=[]
    for table_name in inspector.get_table_names(schema=schema):
        #print("table: %s" % table_name)
        #tables[schema][table_name]=list(inspector.get_columns(table_name, schema=schema)[0].keys())
        tables[schema].append(table_name)

In [10]:
#productos que se están trabajando
productos=pd.read_sql_table('min_ciencias_products', con=cnx,schema='tracking')['0'].to_list()

In [11]:
productos=['producto50','producto53']

In [12]:
tbs={}
for p in productos:
    tbs[p]=tables[p]
tables=tbs

In [13]:
#información comunal
INFO_COMUNAL=pd.read_sql_table('info_comunal@ivanMSC', con=cnx,schema='@ivanMSC') 
INFO_COMUNAL=INFO_COMUNAL.rename(columns={'Nombre':'Comuna'})
INFO_COMUNAL['Codigo region']=INFO_COMUNAL['Codigo region'].astype(int)

In [14]:
#códigos iso
iso=pd.read_sql_table('iso', con=cnx,schema='geo') 
iso=iso.loc[iso['Country Name']=='Chile',('Country Code','Subdivision Name Used','Code','Number',
                                      'Latitude','Longitude')]
iso['Number']=iso['Number'].apply(lambda x: roman.fromRoman(x))
iso=iso.rename(columns={'Number':'Codigo region','Subdivision Name Used':'Region'})
#población regional
popREG=pd.DataFrame(INFO_COMUNAL.groupby('Codigo region')['Poblacion'].sum().reset_index())
#apendizar población regional a códigos ISO
iso=iso.merge(popREG,on='Codigo region')
iso=iso.drop('Region',axis=1)

In [15]:
#defunciones deis comunal (confirmado y sospechoso)
x=pd.read_sql_table('producto50_DefuncionesDEIS_confirmadosPorComuna_std', con=cnx,schema='producto50')
x['Status']='Confirmado'
y=pd.read_sql_table('producto50_DefuncionesDEIS_sospechososPorComuna_std', con=cnx,schema='producto50')
y['Status']='Sospechoso'
x=pd.concat([x,y])
x=x.fillna(0)
x=x[x['Comuna']!='Total']

In [16]:
derived_tables={}
derived_tables['producto50']={}
derived_tables['producto50']['producto50_DefuncionesDEIS_consolidadoPorComuna_std']=x
for s in tables.keys():
    for n in tables[s]:
        #print(n)
        if 'cumulativo' in n.lower():
            print('reading table '+n+'from schema '+s)
            table=pd.read_sql_table(n, con=cnx,schema=s) 
            print('generating diff table for cumulative table: '+n)
            nn=n.lower().replace('cumulativo','_diff_')
            if s not in derived_tables.keys():
                derived_tables[s]={}
            print('new name: '+nn)
            if 'std' not in nn:
                derived_tables[s][nn]=to_diff(table)
            else:
                print('standard form with linear regression')
                derived_tables[s][nn]=to_diff_std(table)
                derived_tables[s][nn]['Fecha']=derived_tables[s][nn]['Fecha'].dt.to_pydatetime()
        
        elif 'producto53' in n.lower():
            print('reading table '+n+' from schema '+s)
            table=pd.read_sql_table(n, con=cnx,schema=s) 
            if s not in derived_tables.keys():
                derived_tables[s]={}
                
            print('adding descriptive stats to table: '+n)
            derived_tables[s][n]=casos_nuevos_desc(table,numeric_col_string='confirmados',group_col='Region')
            
            nn=n.lower()+'_casos_activos'
            #nn=n
            
            if ('producto53_confirmados_regionale' in n.lower()) & ('_casos_activos' not in n.lower()):
                print('calculando casos activos con rolling avg de 14 días')
                print('new name: '+nn)
                derived_tables[s][n]=derived_tables[s][n].merge(iso,on='Codigo region',how='left')
                derived_tables[s][nn]=casos_activos(table,numeric_col_string='confirmados',group_col='Region',window=14)
                derived_tables[s][nn]=derived_tables[s][nn].merge(iso,on='Codigo region',how='left')
                derived_tables[s][nn]=reverse_idx(derived_tables[s][nn],date_field='fecha')
                
            elif ('producto53_confirmados_nacional' in n.lower()) & ('_casos_activos' not in n.lower()):
                print('calculando casos activos con rolling avg de 14 días')
                print('new name: '+nn)
                derived_tables[s][nn]=casos_activos(table,numeric_col_string='confirmados',group_col=None,window=14)
            #elif ('confirmados_provinciale' in n.lower()) & ('_casos_activos' not in n.lower()):
            #    print('calculando casos activos con rolling avg de 14 días')
            #    print('new name: '+nn)
            #    derived_tables[s][nn]=casos_activos(table,numeric_col_string='confirmados',group_col='Provincia',window=14)  
            #elif ('producto53_confirmados_' in n.lower()) & ('_casos_activos' not in n.lower()):
            #    print('calculando casos activos con rolling avg de 14 días')
            #    print('new name: '+nn)
            #    derived_tables[s][nn]=casos_activos(table,numeric_col_string='confirmados',group_col='servicio.salud',window=14)            

reading table producto53_confirmados_regionale from schema producto53
adding descriptive stats to table: producto53_confirmados_regionale
calculando casos activos con rolling avg de 14 días
new name: producto53_confirmados_regionale_casos_activos


/home/jovyan/work/ETL/../modules/transforms.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['Promedio']=nuevos.mean(axis=1)
/home/jovyan/work/ETL/../modules/transforms.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['Min']=nuevos.min(axis=1)
/home/jovyan/work/ETL/../modules/transforms.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

reading table producto53_confirmados_ from schema producto53
adding descriptive stats to table: producto53_confirmados_
reading table producto53_confirmados_nacional from schema producto53


/home/jovyan/work/ETL/../modules/transforms.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['Promedio']=nuevos.mean(axis=1)
/home/jovyan/work/ETL/../modules/transforms.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['Min']=nuevos.min(axis=1)
/home/jovyan/work/ETL/../modules/transforms.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

adding descriptive stats to table: producto53_confirmados_nacional
calculando casos activos con rolling avg de 14 días
new name: producto53_confirmados_nacional_casos_activos
reading table producto53_confirmados_nacional_casos_activos from schema producto53
adding descriptive stats to table: producto53_confirmados_nacional_casos_activos


/home/jovyan/work/ETL/../modules/transforms.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['Promedio']=nuevos.mean(axis=1)
/home/jovyan/work/ETL/../modules/transforms.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['Min']=nuevos.min(axis=1)
/home/jovyan/work/ETL/../modules/transforms.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

reading table producto53_confirmados_provinciale from schema producto53
adding descriptive stats to table: producto53_confirmados_provinciale


In [17]:
len(derived_tables)

2

In [18]:
list(derived_tables.keys())

['producto50', 'producto53']

In [19]:
path

'/data/ETLcache/'

In [31]:
for schema in ['producto53']:
    for name in derived_tables[schema].keys():
        try:
            df=derived_tables[schema][name]
            if 'level_0' in df.columns:
                df=df.drop('level_0',axis=1)
            name=name.replace('-','_')
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
            print("saving table"+path+name+timestamp+'.csv in cache')
            df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
        except Exception as e: 
            print(str(e))
            pass

creating table producto53_confirmados_regionale ,schema: producto53
saving table/data/ETLcache/producto53_confirmados_regionale_06122021_014153.csv in cache
creating table producto53_confirmados_regionale_casos_activos ,schema: producto53
saving table/data/ETLcache/producto53_confirmados_regionale_casos_activos_06122021_014153.csv in cache
creating table producto53_confirmados_ ,schema: producto53
saving table/data/ETLcache/producto53_confirmados__06122021_014153.csv in cache
creating table producto53_confirmados_nacional ,schema: producto53
saving table/data/ETLcache/producto53_confirmados_nacional_06122021_014153.csv in cache
creating table producto53_confirmados_nacional_casos_activos ,schema: producto53
saving table/data/ETLcache/producto53_confirmados_nacional_casos_activos_06122021_014153.csv in cache
creating table producto53_confirmados_provinciale ,schema: producto53
saving table/data/ETLcache/producto53_confirmados_provinciale_06122021_014153.csv in cache


In [33]:
del(derived_tables['producto53'])

In [34]:
derived_tables.keys()

dict_keys(['producto50'])

In [35]:
for schema in ['producto50']:
    for name in derived_tables[schema].keys():
        try:
            df=derived_tables[schema][name]
            if 'level_0' in df.columns:
                df=df.drop('level_0',axis=1)
            name=name.replace('-','_')
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
            print("saving table"+path+name+timestamp+'.csv in cache')
            df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
        except Exception as e: 
            print(str(e))
            pass

creating table producto50_DefuncionesDEIS_consolidadoPorComuna_std ,schema: producto50
saving table/data/ETLcache/producto50_DefuncionesDEIS_consolidadoPorComuna_std_06122021_014153.csv in cache


In [29]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile-derived.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile-derived@min-ciencias.ipynb')

65280